In [1]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_theme()

import torch 
import torch.nn as nn 
import torch.optim as optim


from models.learned_qp_solver_1 import MLP_Init, learned_qp_solver
from scipy.io import loadmat
from torch.utils.data import Dataset, DataLoader
# torch.set_default_dtype(torch.float64)

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cuda device


In [3]:


num = 50
t = 0.05


vel_min = -1.5
vel_max = 1.5

acc_min = -3.0
acc_max = 3.0

jerk_min = -3.0
jerk_max = 3.0

theta_min = -3.14
theta_max = 3.14



In [4]:
#### creating dataset 

dataset_size = 200000

vel_samples = np.random.uniform(3*vel_min, 3*vel_max,  (dataset_size, num)    )

theta_init = np.random.uniform(theta_min, theta_max, (dataset_size, 1)  )
vel_init = np.random.uniform(vel_min, vel_max, (dataset_size, 1)  )
acc_init = np.random.uniform(acc_min, acc_max, (dataset_size, 1)  )


inp = np.hstack(( vel_samples, theta_init, vel_init ))

# normalization_const = (vel_max-vel_min)

inp_mean, inp_std = inp.mean(), inp.std()



print(inp_mean, inp_std)



0.0005092294846281164 2.5628748727074853


In [5]:
# Custom Dataset Loader 
class TrajDataset(Dataset):
	"""Expert Trajectory Dataset."""
	def __init__(self, inp, vel_init, acc_init, theta_init, vel_samples):
		
		# State Data
		self.inp = inp
  
		self.vel_init = vel_init
		self.acc_init = acc_init
		
		# PCD Data
		self.theta_init = theta_init
		
		# Expert Coeff
		self.vel_samples = vel_samples

		
		
	
	def __len__(self):
		return len(self.inp)    
			
	def __getitem__(self, idx):
		
		# Inputs
		inp = self.inp[idx]
		vel_init = self.vel_init[idx]

		acc_init = self.acc_init[idx]
		theta_init = self.theta_init[idx]
		
		# Outputs
		vel_samples = self.vel_samples[idx]
	
		
		
		return torch.tensor(inp).float(), torch.tensor(vel_init).float(), torch.tensor(acc_init).float(), torch.tensor(theta_init).float(), torch.tensor(vel_samples).float()
	

# Batch Size - 3k or 4k
batch_size = 2056

# pcd_data = pcd_data.reshape(data_set_size, 2, 200)

# Using PyTorch Dataloader
train_dataset = TrajDataset(inp, vel_init, acc_init,  theta_init, vel_samples)
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=0, drop_last=True)



In [6]:
# Differentiable Layer
num_batch = train_loader.batch_size


mlp_init_inp_dim = np.shape(inp)[1]
mlp_init_hidden_dim = 1024
# mlp_init_out_dim = (2*nvar)**2+2*nvar+2*nvar
mlp_init_out_dim = 2*num+(2*num+2*num+2*(num-1)+2*(num-2))

# mlp_pred = MLP_Pred(mlp_pred_inp_dim, mlp_pred_hidden_dim, mlp_pred_out_dim  )
mlp_init = MLP_Init(mlp_init_inp_dim, mlp_init_hidden_dim, mlp_init_out_dim  )


model = learned_qp_solver(num_batch, num, t, mlp_init, inp_mean, inp_std).to(device)
# model.load_state_dict(torch.load('./weights/learned_qp_1.pth'))



In [7]:
epochs = 800
step = 0 
beta = 1.0 # 3.5

optimizer = optim.AdamW(model.parameters(), lr = 1e-3, weight_decay=6e-5)
# optimizer = optim.RMSprop(model.parameters(), lr = 1e-3, weight_decay=6e-5)

# scheduler = optim.lr_scheduler.StepLR(optimizer, step_size = 100, gamma = 0.1)

avg_train_loss, avg_loss_primal, avg_fixed_point_loss, avg_proj_loss = [], [], [], []
model.train()
for epoch in range(epochs):
	
	# Train Loop
	losses_train, aug_losses_primal, aug_losses_fixed_point, aug_losses_proj = [], [], [], []
	
	for (inp, vel_init, acc_init,  theta_init, vel_samples) in train_loader:
		
		# Input and Output 
  
		################################################################################################

		# print(goal_des[0])
		# print(y_lb[0], y_ub[0])

  
		inp = inp.to(device)
		vel_init = vel_init.to(device)
		acc_init = acc_init.to(device)
		theta_init = theta_init.to(device)

		vel_samples = vel_samples.to(device)
	
		
		vel_projected, res_primal_stack, res_fixed_point_stack, accumulated_res_primal, accumulated_res_fixed_point = model(inp, vel_init, acc_init, vel_samples, vel_max, vel_min, acc_max, acc_min, jerk_max, jerk_min, theta_min, theta_max, theta_init)
		loss, primal_loss, fixed_point_loss, proj_loss = model.ss_loss(accumulated_res_primal, accumulated_res_fixed_point, vel_projected, vel_samples)
	
		optimizer.zero_grad()
		loss.backward()
		# torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=0.5)
		optimizer.step()
		
		losses_train.append(loss.detach().cpu().numpy()) 
		aug_losses_primal.append(primal_loss.detach().cpu().numpy())
		aug_losses_fixed_point.append(fixed_point_loss.detach().cpu().numpy())
		aug_losses_proj.append(proj_loss.detach().cpu().numpy())
		# aug_losses_steer.append(steer_loss.detach().cpu().numpy())

		
	# scale = scale*1.2	
		
	if epoch % 4 == 0:    
		print(f"Epoch: {epoch + 1}, Train Loss: {np.average(losses_train):.3f}, aug_loss_primal: {np.average(aug_losses_primal):.3f}, aug_loss_fixed_point: {np.average(aug_losses_fixed_point):.3f}, aug_loss_proj: {np.average(aug_losses_proj):.3f} ")

	step += 0.15 #0.15
	# scheduler.step()
	avg_train_loss.append(np.average(losses_train)), avg_loss_primal.append(np.average(aug_losses_primal)), avg_fixed_point_loss.append(np.average(aug_losses_fixed_point)),  avg_proj_loss.append(np.average(aug_losses_proj))
	
		



Epoch: 1, Train Loss: 4.647, aug_loss_primal: 0.010, aug_loss_fixed_point: 1.242, aug_loss_proj: 3.405 


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), './weights/learned_qp_1.pth')